In [0]:
from pyspark.sql import functions as F, Window
from delta.tables import DeltaTable
from py_functions_silver import *
from pyspark.sql.functions import current_timestamp

BRONZE = "uc_athlete_data.bronze.strava_activities"
SILVER = "uc_athlete_data.silver.strava_activities"
CHECKPOINT = "abfss://silver@adlsathlete.dfs.core.windows.net/strava/activities/activities_checkpoint/"

# chave(s) de negócio — mude p/ composto se precisar, ex.: ["activity_id","lap_index"]
BUSINESS_KEYS = ["id"]
# colunas de ordenação para decidir o “vencedor” nos duplicados do micro-lote
ORDER_COLS = ["ingestion_timestamp"]  # adicione "updated_at" se existir

In [0]:
from pyspark.sql import functions as F

REJECT_TABLE = "uc_athlete_data.silver_rejects.strava_activities"

def assert_quality(df):
    # 1) Defina as regras (nome -> condição SQL)
    rules = {
        # obrigatórios
        "missing_id":               "id IS NULL",
        "missing_start_date":       "start_date IS NULL",
        "sport_type_missing":       "sport_type IS NULL OR sport_type = ''",

        # distância/tempo básicos
        "neg_distance":             "distance_km < 0",
        "zero_distance_run":        "sport_type = 'Run' AND distance_km = 0",
        #"nonpositive_times":        "elapsed_time <= 0 OR moving_time <= 0",
        #"moving_gt_elapsed":        "moving_time > elapsed_time",

        # velocidade
        "speed_negative":           "average_speed_kmh < 0",
        "speed_implausible_run":    "sport_type = 'Run' AND average_speed_kmh > 30",
        #"speed_implausible_ride":   "sport_type IN ('Ride','VirtualRide') AND average_speed_kmh > 80",

        # coerência vel = dist / tempo (tolerância ~1 km/h)
        #"speed_inconsistent":       "moving_time > 0 AND distance_km >= 0 "
        #                            "AND abs(average_speed_kmh - (distance_km / (moving_time/3600.0))) > 1.0",

        # pace calculado
        "pace_negative":            "pace_min_km_new < 0",
        "pace_null_when_should":    "moving_time > 0 AND distance_km > 0 AND pace_min_km_new IS NULL",

        # elevação
        #"gain_negative":            "total_elevation_gain < 0",

        # vitais / cadência (se presentes)
        #"hr_out_of_range":          "average_heartrate IS NOT NULL AND (average_heartrate < 30 OR average_heartrate > 230)",
        #"cadence_out_of_range":     "average_cadence IS NOT NULL AND (average_cadence < 0 OR average_cadence > 300)",
    }

    # 2) Constrói uma coluna array com TODOS os motivos que baterem
    #    (evita múltiplos scans/union e duplicaçāo na rejects)
    #dfq = df.withColumn("reject_reasons", F.array())  # array<string> inicial
    dfq = df.withColumn("reject_reasons", F.lit(None).cast("array<string>"))
    
    for name, sql_cond in rules.items():
        cond_col = F.expr(sql_cond)
        dfq = dfq.withColumn(
            "reject_reasons",
            F.when(
                cond_col,
                # coalesce para lidar com NULL -> vira [] antes de unir com [name]
                F.array_union(F.coalesce(F.col("reject_reasons"), F.array()), F.array(F.lit(name)))
            ).otherwise(F.col("reject_reasons"))
        )

    # 3) Flag, contador e string consolidada (para leitura humana)
    dfq = (dfq
           .withColumn("reject_reason_count",
                       F.when(F.col("reject_reasons").isNull(), F.lit(0))
                        .otherwise(F.size("reject_reasons")))
           .withColumn("reject_reason",
                       F.when(F.col("reject_reasons").isNull(), F.lit(None).cast("string"))
                        .otherwise(F.array_join("reject_reasons", "|")))
           .withColumn("reject_timestamp", F.current_timestamp())
          )

    # 4) Separa válidos x rejeitos (cada registro entra UMA vez na rejects)
#    df_reject = dfq.filter(F.col("reject_reason_count") > 0 )
#    df_valid  = dfq.filter(F.col("reject_reason_count") == 0).drop("reject_reasons","reject_reason","reject_reason_count","reject_timestamp")
    #display(dfq)
    df_reject = dfq.filter(F.col("reject_reason").isNotNull())
    #display(df_reject)
    df_valid  = dfq.filter(F.col("reject_reason").isNull()).drop("reject_reasons","reject_reason","reject_reason_count","reject_timestamp")

    # 5) Persiste rejects (append) — sem duplicação da mesma linha
    #    Obs: não precisa mergeSchema aqui porque já criamos a tabela com colunas extras.
    if df_reject:  # evita ação pesada; é um job pequeno
        (df_reject
        .write
        .format("delta")        
        .mode("append")
        #.option("checkpointLocation", "/mnt/checkpoints/rejects")
        .option("mergeSchema", "true")
        .saveAsTable(REJECT_TABLE)
        #.start()
        )
        print(f"⚠️ {df_reject.count()} registros rejeitados salvos em {REJECT_TABLE}")

    return df_valid


In [0]:
def _dedupe_microbatch(df):
    w = Window.partitionBy(*[F.col(c) for c in BUSINESS_KEYS]) \
         .orderBy(*[F.col(c).desc() for c in ORDER_COLS])
    return (df.withColumn("row_id", F.row_number().over(w))
              .filter(F.col("row_id")==1)
              .drop("row_id"))

In [0]:
# --- Compose (aplica tudo que você mostrou) -----------------------------------

def apply_all_silver_calcs(df: DataFrame,
                           *,
                           type_col: str = "type",
                           distance_col: str = "distance",
                           average_speed_col: str = "average_speed",
                           moving_time_col: str = "moving_time",
                           elapsed_time_col: str = "elapsed_time",
                           start_date_col: str = "start_date",
                           non_run_value_for_pace=0  # para manter igual ao seu snippet
                           ) -> DataFrame:
    """
    Aplica TODAS as transformações do snippet original.
    Retorna um novo DataFrame com:
      - start_date (date)
      - distance_km
      - average_speed_kmh
      - pace_min_km
      - pace_min_km_moving_time
      - tempo_real (HH:MM:SS)
      - pace_min_km_new
      - pace_strava (M:SS)
      - dia_semana
    """
    return (
        df
        .transform(lambda d: add_start_date(d, src_col=start_date_col, out_col="start_date"))
        .transform(lambda d: add_distance_km(d, distance_col=distance_col, out_col="distance_km", decimals=2))
        .transform(lambda d: add_average_speed_kmh(d, avg_speed_col=average_speed_col, out_col="average_speed_kmh", decimals=2))
        .transform(lambda d: add_pace_min_km(d,
                                             elapsed_time_col=elapsed_time_col,
                                             distance_col=distance_col,
                                             type_col=type_col,
                                             out_col="pace_min_km",
                                             decimals=2,
                                             only_for_run=True,
                                             non_run_value=non_run_value_for_pace))
        .transform(lambda d: add_pace_min_km_moving_time(d,
                                                         moving_time_col=moving_time_col,
                                                         distance_col=distance_col,
                                                         type_col=type_col,
                                                         out_col="pace_min_km_moving_time",
                                                         decimals=2,
                                                         only_for_run=True,
                                                         non_run_value=non_run_value_for_pace))
        .transform(lambda d: add_tempo_real(d, seconds_col=moving_time_col, out_col="tempo_real"))
        .transform(lambda d: add_pace_min_km_new(d,
                                                 moving_time_col=moving_time_col,
                                                 distance_col=distance_col,
                                                 out_col="pace_min_km_new",
                                                 decimals=3))
        .transform(lambda d: add_pace_strava(d, pace_min_col="pace_min_km_new", out_col="pace_strava"))
        .transform(lambda d: add_dia_semana(d, date_col="start_date", out_col="dia_semana", pattern="E"))
    )

In [0]:
%sql
SELECT count(*) as total, reject_reason, date(reject_timestamp) as dt
FROM uc_athlete_data.silver_rejects.strava_activities
GROUP BY 2,3
ORDER BY 3 DESC;


In [0]:
def upsert_data(microBatchDF, batch):
    microBatchDF.createOrReplaceTempView("activities_microbatch")
    
    sql_query = """
                MERGE INTO uc_athlete_data.silver.strava_activities A
                USING activities_microbatch B
                ON A.ID = b.ID AND A.INGESTION_TIMESTAMP = B.INGESTION_TIMESTAMP
                WHEN NOT MATCHED THEN INSERT * 
                """  

    microBatchDF.sparkSession.sql(sql_query)

In [0]:
def load_data(microBatchDF, batch):   
    
    microBatchDF = _dedupe_microbatch(microBatchDF)
    print("dedup ok ")
    microBatchDF = apply_all_silver_calcs(microBatchDF)
    print("calcs ok ")
    df_clean = assert_quality(microBatchDF)
    print("clean ok ")
    upsert_data(df_clean, batch)
    print("upsert ok ")

In [0]:
bronze_stream = spark.readStream.table(BRONZE)
query = (bronze_stream.writeStream
                 .foreachBatch(load_data)
                 .option("checkpointLocation", CHECKPOINT)
                 .trigger(availableNow=True)
                 .start())
                 
query.awaitTermination()